In [ ]:
#default_exp data.datasets.on

In [ ]:
#export
from dataclasses import dataclass
from datetime import timedelta
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
from pandas.tseries.frequencies import to_offset

from nixtla.data.datasets.utils import download_file, Info, TimeSeriesDataclass
from nixtla.data.ontsdataset import TimeSeriesDataset

In [ ]:
#export
def load_on_data(root_dir, test_date, last_date):
    # Declare future variables
    f_cols = ['PD_0','PS_0','day_0','day_1','day_2','day_3','day_4','day_5','day_6',
              'hour_0', 'hour_1','hour_2','hour_3','hour_4','hour_5','hour_6','hour_7','hour_8',
              'hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16',
              'hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23',
              'temperature_0', 'temperature_1', 'temperature_2', 'temperature_3', 'temperature_4',
              'pressure_0', 'pressure_1', 'pressure_2', 'pressure_3', 'pressure_4', 'humidity_0',
              'humidity_1', 'humidity_2', 'humidity_3', 'humidity_4', 'wind_speed_0', 'wind_speed_1',
              'wind_speed_2', 'wind_speed_3', 'wind_speed_4', 'rain_0', 'rain_1', 'rain_2', 'rain_3', 'rain_4', 'total_demand']

    # Demanda Final
    demanda_final = pd.read_csv(f'{root_dir}/Demanda_Final.csv')
    demanda_final.columns = ['ds','y']
    demanda_final['unique_id'] = 'demanda_final'
    demanda_final['ds'] = pd.to_datetime(demanda_final['ds'])
    demanda_final = demanda_final[demanda_final['ds']<last_date]
    demanda_final = demanda_final[demanda_final['ds']>='2017-10-07'].reset_index(drop=True)

    # Ejecutado
    ejecutado = pd.read_csv(f'{root_dir}/Despacho_Reprograma.csv')
    ejecutado = ejecutado[['DATE_TIME','EJECUTADO']]
    ejecutado.columns = ['ds','ejecutado']
    ejecutado['ds'] = pd.to_datetime(ejecutado['ds'])
    ejecutado = ejecutado[ejecutado['ds']<last_date]
    ejecutado = ejecutado[ejecutado['ds']>='2017-10-07'].reset_index(drop=True)

    # Train data (ON exogenous variables)
    train_data = pd.read_csv(f'{root_dir}/Training_Data.csv')
    train_data = train_data.rename(columns={'CURRENT_TIME':'ds'})
    train_data['ds'] = pd.to_datetime(train_data['ds'])
    train_data = train_data[train_data['ds']<last_date]
    train_data = train_data[train_data['ds']>='2017-10-07'].reset_index(drop=True)

    # Historic Weather
    data_weather = pd.read_csv(f'{root_dir}/HistoricalWeather.csv')
    data_weather['dt_iso'] = data_weather['dt_iso'].str[:-10]
    data_weather['dt_iso'] = pd.to_datetime(data_weather['dt_iso'])
    data_weather = data_weather[data_weather['dt_iso']>='2017-10-07'].reset_index(drop=True)
    data_weather = data_weather[['dt_iso', 'city_name', 'temp', 'feels_like', 'rain_1h', 'pressure', 'humidity', 'wind_speed', 'clouds_all']]
    data_weather = data_weather.drop_duplicates(subset=['dt_iso', 'city_name']).reset_index(drop=True)
    data_weather = data_weather.pivot(index='dt_iso', columns='city_name').reset_index()
    data_weather.columns = ['_'.join(col).strip() for col in data_weather.columns.values]
    data_weather = data_weather.rename(columns = {'dt_iso_':'ds'})
    data_weather = data_weather.fillna(0) # Fill rain nans with 0

    # Forecast Weather
    data_f_weather = pd.read_csv(f'{root_dir}/Historical ForecastWeather.csv')
    # Between 1 and 21,600 takes next 6h of forecasts. Always uses last available forecast for each ds
    data_f_weather = data_f_weather[(data_f_weather['slice dt unixtime']-data_f_weather['forecast dt unixtime']). \
                                    between(1, 21600)].reset_index(drop=True)
    data_f_weather['forecast dt iso'] = data_f_weather['forecast dt iso'].str[:-10]
    data_f_weather['forecast dt iso'] = pd.to_datetime(data_f_weather['forecast dt iso'])
    data_f_weather = data_f_weather[data_f_weather['forecast dt iso']>='2017-10-07'].reset_index(drop=True)
    data_f_weather = data_f_weather.rename(columns = {'forecast dt iso':'ds'})
    data_f_weather['city'] = data_f_weather.groupby(['lat']).ngroup()
    data_f_weather['city'] = data_f_weather['city'].astype(str)
    data_f_weather = data_f_weather[['ds', 'city', 'temperature', 'pressure', 'humidity', 'wind_speed', 'rain']]
    data_f_weather = data_f_weather.drop_duplicates(subset=['ds', 'city']).reset_index(drop=True)
    data_f_weather = data_f_weather.pivot(index='ds', columns='city').reset_index()
    data_f_weather.columns = ['_'.join(col).strip() for col in data_f_weather.columns.values]
    data_f_weather = data_f_weather.rename(columns = {'ds_':'ds'})
    data_f_weather = data_f_weather.fillna(0) # Fill rain nans with 0

    # Total demand of large clients
    large_demand = pd.read_csv(f'{root_dir}/usuarios_libres_demanda_prevista_semanal.csv')
    large_demand['EQUIPO'] = pd.to_datetime(large_demand['EQUIPO'])
    large_demand = large_demand[large_demand['EQUIPO']>='2017-10-07'].reset_index(drop=True)
    large_demand = large_demand.fillna(0)
    large_demand['total_demand'] = np.sum(large_demand.values[:,1:], axis=1)
    large_demand = large_demand[['EQUIPO', 'total_demand']]
    large_demand.columns = ['ds', 'total_demand']

    # Merge
    data = demanda_final.merge(ejecutado, how='left', indicator=False) #ejecutado needs to be first exogenous variable
    data = data.merge(train_data, how='left', indicator=False)
    data[['ejecutado']] = data[['ejecutado']].fillna(method='ffill') #ffill to avoid leakage from immediate next ejecutado
    data = data.merge(data_weather, how='left', on='ds')
    data = data.merge(data_f_weather, how='left', on='ds')
    data = data.merge(large_demand, how='left', on='ds')
    data = data.fillna(method='ffill') # ffill 15-mintues with last available forecast

    data['hour'] = data['ds'].dt.hour
    dummies = pd.get_dummies(data['hour'],prefix='hour')
    data = pd.concat((data,dummies), axis=1)

    data['day'] = data['ds'].dt.dayofweek
    dummies = pd.get_dummies(data['day'],prefix='day')
    data = pd.concat((data,dummies), axis=1)

    dummies_cols = [col for col in data if (col.startswith('day') or col.startswith('hour_'))]

    X_t_df = data.drop(columns=['hour','day','y'])
    y_df = data[['unique_id','ds','y']]

    Y_insample_df = y_df[y_df['ds'] < test_date].reset_index(drop=True)
    Y_outsample_df = y_df[y_df['ds'] >= test_date].reset_index(drop=True)

    X_insample_df = X_t_df[X_t_df['ds'] < test_date].reset_index(drop=True)
    X_outsample_df = X_t_df[X_t_df['ds'] >= test_date].reset_index(drop=True)

    # Filter_variables
    variables = ['ds','unique_id','ejecutado', 'total_demand', 'PD_0', 'DECIMAL_HOUR',
                 'temperature_0', 'temperature_1', 'temperature_2', 'temperature_3', 'temperature_4',
                 'pressure_0', 'pressure_1', 'pressure_2', 'pressure_3', 'pressure_4',
                 'humidity_0', 'humidity_1', 'humidity_2', 'humidity_3', 'humidity_4',
                 'rain_0', 'rain_1', 'rain_2', 'rain_3', 'rain_4']

    X_insample_df = X_insample_df[variables] 
    X_outsample_df = X_outsample_df[variables] 
    return Y_insample_df, X_insample_df, Y_outsample_df, X_outsample_df, f_cols

In [ ]:
Y_df, X_df, Y_outsample_df, X_outsample_df, f_cols = load_on_data(root_dir='../data/on',
                                                                  test_date='2020-01-01',
                                                                  last_date='2021-01-01')

In [ ]:
Y_df.head()

,unique_id,ds,y
0,demanda_final,2017-10-07 00:00:00,5545.98338
1,demanda_final,2017-10-07 00:15:00,5448.66753
2,demanda_final,2017-10-07 00:30:00,5392.92989
3,demanda_final,2017-10-07 00:45:00,5371.67098
4,demanda_final,2017-10-07 01:00:00,5296.58436


In [ ]:
X_df.head()

,ds,unique_id,ejecutado,total_demand,PD_0,DECIMAL_HOUR,temperature_0,temperature_1,temperature_2,temperature_3,temperature_4,pressure_0,pressure_1,pressure_2,pressure_3,pressure_4,humidity_0,humidity_1,humidity_2,humidity_3,humidity_4,rain_0,rain_1,rain_2,rain_3,rain_4
0,2017-10-07 00:00:00,demanda_final,5522.52359,2236.738,5559.55969,0.00,286.522,289.192,282.922,291.182,279.682,1019.304,1018.519,1020.682,1018.701,1019.444,48.606,56.806,55.206,73.206,50.406,0.0,0.0,0.0,0.0,0.0
1,2017-10-07 00:15:00,demanda_final,5522.52359,2236.738,5375.72396,0.25,286.522,289.192,282.922,291.182,279.682,1019.304,1018.519,1020.682,1018.701,1019.444,48.606,56.806,55.206,73.206,50.406,0.0,0.0,0.0,0.0,0.0
2,2017-10-07 00:30:00,demanda_final,5400.83722,2226.981,5375.72396,0.50,286.522,289.192,282.922,291.182,279.682,1019.304,1018.519,1020.682,1018.701,1019.444,48.606,56.806,55.206,73.206,50.406,0.0,0.0,0.0,0.0,0.0
3,2017-10-07 00:45:00,demanda_final,5400.83722,2226.981,5275.65261,0.75,286.522,289.192,282.922,291.182,279.682,1019.304,1018.519,1020.682,1018.701,1019.444,48.606,56.806,55.206,73.206,50.406,0.0,0.0,0.0,0.0,0.0
4,2017-10-07 01:00:00,demanda_final,5318.78278,2230.466,5275.65261,1.00,286.522,289.192,282.922,291.182,279.682,1019.304,1018.519,1020.682,1018.701,1019.444,48.606,56.806,55.206,73.206,50.406,0.0,0.0,0.0,0.0,0.0
